<a href="https://colab.research.google.com/github/stephenfrein/csc8491/blob/main/DynamoExampleforCSC8491.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
# these are my credentials - you'll need your own in the future 
!mkdir /root/.aws
!touch /root/.aws/credentials
# put the secret line from Blackboard in here
!cat ~/.aws/credentials


In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource('dynamodb', region_name='us-east-2')

In [ ]:
# set your table name
table_name = 'sfrein01_sessions' #something like <your_villanova_username>_sessions - I'll use sfrein01_sessions for mine

table = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'session_id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'session_id',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        # of reads and writes per second until throttling exception returned
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# wait for table to exist
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

# how many rows are in it? remember no data yet
print(table.item_count)

In [ ]:
new_item = table.put_item(
  Item={
        'session_id': 'ABC123XYZ',
        'start_date': '02/18/2021 20:00:00',
        'details': {
                'length_minutes': 60,
                'role': 'ADMIN'
          }
    }
)
print(new_item)

In [ ]:
newer_item = table.put_item(
  Item={
        'session_id': 'DEF321ZYX',
        'start_date': '02/18/2021 21:00:00',
        'details': {
                'length_minutes': 30,
                'role': 'USER'
          }
    }
)
print(newer_item)

In [ ]:
found_session = table.get_item(Key={'session_id': 'ABC123XYZ'})
print(list(found_session.values())[0])

In [ ]:
found_session = table.get_item(Key={'start_date': '02/18/2021 20:00:00'})
print(list(found_session.values())[0])

In [ ]:
client = boto3.client('dynamodb', region_name='us-east-2')

response = client.update_table(
    TableName = table_name,
    AttributeDefinitions=[
            {
                "AttributeName": "start_date",
                "AttributeType": "S"
            },
        ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'StartDateIndex',
                'KeySchema': [
                    {
                        'AttributeName': 'start_date',
                        'KeyType': 'HASH'
                    }
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput': {
                    'ReadCapacityUnits': 1,
                    'WriteCapacityUnits': 1
                }
            }
        }
    ]
)
print(response)

In [ ]:
from boto3.dynamodb.conditions import Key
table = dynamodb.Table(table_name)

# When adding a global secondary index to an existing table, you cannot query the index until it has been backfilled.
# This portion of the script waits until the index is in the “ACTIVE” status, indicating it is ready to be queried.
while True:
    if not table.global_secondary_indexes or table.global_secondary_indexes[0]['IndexStatus'] != 'ACTIVE':
        print('Waiting for index to backfill...')
        time.sleep(5)
        table.reload()
    else:
        break

# When making a Query call, you use the KeyConditionExpression parameter to specify the hash key on which you want to query.
# If you want to use a specific index, you also need to pass the IndexName in our API call.
resp = table.query(
    # Add the name of the index you want to use in your query.
    IndexName="StartDateIndex",
    KeyConditionExpression=Key('start_date').eq('02/18/2021 20:00:00'),
)

print(resp)

In [ ]:
dynamodb.Table(table_name).delete()